<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Recommendation Engines Lesson Exercise Solutions

_Author: Alex Combs (NYC) _

---

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
a = np.array([2, 0, 1, 1, 0, 2, 1, 1])
b = np.array([2, 1, 1, 0, 1, 1, 1, 1])

np.dot(a,b) / (np.sqrt(sum(np.square(a))) * np.sqrt(sum(np.square(b))))

0.8215838362577491

In [5]:
cosine_similarity(a.reshape(1,-1),
                  b.reshape(1,-1))[0][0]

0.8215838362577492

## Exercise: Find the similarity between X and Y and X and Z for the following.

| User |Snarky's Potato Chips	| SoSo Smoth Lotion	|Duffly Beer	|BetterTap Water |XXLargeLivin' Football Jersey	|Snowy Cotton Ballas	|Disposos Diapers|
| - |:-------:| --:| --- | --- | --- | --- | --- |
| X| &nbsp; |4  | &nbsp; | 3  | &nbsp; | 4  | &nbsp;  |
| Y| &nbsp; |3.5| &nbsp; | 2.5| &nbsp; | 4  | 4  |
| Z| &nbsp; |4  | &nbsp; | 3.5| &nbsp; | 4.5| 4.5|

In [8]:
# 1. Vectorize the data

X = np.array([0.0, 4.0, 3.0, 0.0, 4.0, 0.0])
Y = np.array([0.0, 3.5, 2.5, 0.0, 4.0, 4.0])
Z = np.array([0.0, 4.0, 3.5, 0.0, 4.5, 4.5])

# 2. Mean-center

X[X!=0] -= np.mean(X[X!=0])
Y[Y!=0] -= np.mean(Y[Y!=0])
Z[Z!=0] -= np.mean(Z[Z!=0])

# 3. Cosine similarity

from sklearn.metrics.pairwise import cosine_similarity

print("similarity(X, Y) = " + str(cosine_similarity(X.reshape(1,-1), Y.reshape(1,-1))))
print("similarity(Y, Z) = " + str(cosine_similarity(Y.reshape(1,-1), Z.reshape(1,-1))))
print("similarity(X, Z) = " + str(cosine_similarity(X.reshape(1,-1), Z.reshape(1,-1))))

similarity(X, Y) = [[0.83333333]]
similarity(Y, Z) = [[0.98473193]]
similarity(X, Z) = [[0.73854895]]


## Independent Exercise:

Write a function that takes in a utility matrix with users along the index and songs along the columns as seen above in the item-to-item filtering example. The function should accept a target user and song (as strings) that it will return a rating for. 

Use the following as your utility matrix:

In [10]:
df = pd.DataFrame({'U1':[2 , None, 1, None, 3], 
                   'U2': [None, 3, None, 4,None],
                   'U3': [4, None, 5, 4, None], 
                   'U4': [None, 3, None, 4, None], 
                   'U5': [5, None, 4, None, 5]},
                 index = ['S1', 'S2', 'S3', 'S4', 'S5'])

In [11]:
df

,U1,U2,U3,U4,U5
S1,2.0,NaN,4.0,NaN,5.0
S2,NaN,3.0,NaN,3.0,NaN
S3,1.0,NaN,5.0,NaN,4.0
S4,NaN,4.0,4.0,4.0,NaN
S5,3.0,NaN,NaN,NaN,5.0


This is not the most elegant solution - should really accept a k value, but here it is fixed at 2

In [12]:
def get_rating(df, user, song):
    def ret_sim(x):
        return cosine_similarity(np.array(mc_df.loc[song]).reshape(1,-1), np.array(x).reshape(1,-1))[0][0]
    
    mc_df = df.apply(lambda x: x - np.mean(x), axis=1).fillna(0)
    mc_df['co_sim'] = mc_df.apply(ret_sim, axis=1)
    print(mc_df)
    
    two_nearest_songs = mc_df.sort_values('co_sim', ascending=False)[1:3]['co_sim'].index.values
    two_nearest_cosims = mc_df.sort_values('co_sim', ascending=False)[1:3]['co_sim'].values
    two_nearest_ratings = df.loc[two_nearest_songs][user].values
    
    return ((two_nearest_cosims[0] * two_nearest_ratings[0]) + (two_nearest_cosims[1] * two_nearest_ratings[1]))/sum(two_nearest_cosims)

In [13]:
get_rating(df, 'U3', 'S5')

          U1   U2        U3   U4        U5    co_sim
S1 -1.666667  0.0  0.333333  0.0  1.333333  0.981981
S2  0.000000  0.0  0.000000  0.0  0.000000  0.000000
S3 -2.333333  0.0  1.666667  0.0  0.666667  0.720577
S4  0.000000  0.0  0.000000  0.0  0.000000  0.000000
S5 -1.000000  0.0  0.000000  0.0  1.000000  1.000000


4.423232002361576